# Úvod do pravděpodobnosti a statistiky
V tomto zápisníku si budeme hrát s některými koncepty, které jsme dříve probírali. Mnoho konceptů z pravděpodobnosti a statistiky je dobře zastoupeno v hlavních knihovnách pro zpracování dat v Pythonu, jako jsou `numpy` a `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Náhodné proměnné a rozdělení
Začněme výběrem vzorku 30 hodnot z rovnoměrného rozdělení od 0 do 9. Také vypočítáme průměr a rozptyl.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Pro vizuální odhad, kolik různých hodnot se ve vzorku vyskytuje, můžeme nakreslit **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Analýza reálných dat

Průměr a rozptyl jsou velmi důležité při analýze reálných dat. Načtěme data o baseballových hráčích z [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Používáme balíček zvaný [**Pandas**](https://pandas.pydata.org/) pro analýzu dat. O knihovně Pandas a práci s daty v Pythonu budeme mluvit později v tomto kurzu.

Spočítejme průměrné hodnoty pro věk, výšku a váhu:


In [ ]:
df[['Age','Height','Weight']].mean()

Nyní se zaměřme na výšku a vypočítejme směrodatnou odchylku a rozptyl:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Kromě průměru dává smysl podívat se na medián a kvartily. Ty lze vizualizovat pomocí **krabicového grafu**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Můžeme také vytvořit krabicové grafy podmnožin našeho datasetu, například seskupené podle role hráče.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Poznámka**: Tento diagram naznačuje, že průměrná výška prvních basemanů je vyšší než výška druhých basemanů. Později se naučíme, jak tuto hypotézu formálněji otestovat a jak ukázat, že naše data jsou statisticky významná k jejímu prokázání.

Věk, výška a váha jsou všechny spojité náhodné proměnné. Jaká myslíte, že je jejich distribuce? Dobrou metodou, jak to zjistit, je vykreslit histogram hodnot:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normální rozdělení

Vytvořme umělý vzorek váh, který následuje normální rozdělení se stejným průměrem a rozptylem jako naše skutečná data:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Protože většina hodnot v reálném životě má normální rozdělení, neměli bychom používat uniformní generátor náhodných čísel pro generování vzorových dat. Zde je, co se stane, pokud se pokusíme vygenerovat hmotnosti s uniformním rozdělením (vygenerované pomocí `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervaly spolehlivosti

Nyní vypočítáme intervaly spolehlivosti pro váhy a výšky baseballových hráčů. Použijeme kód [z této diskuse na stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Testování hypotéz

Pojďme prozkoumat různé role v našem datasetu baseballových hráčů:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Ověřme hypotézu, že první metaři jsou vyšší než druzí metaři. Nejjednodušší způsob, jak to udělat, je otestovat intervaly spolehlivosti:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Vidíme, že intervaly se nepřekrývají.

Statisticky správnějším způsobem, jak dokázat hypotézu, je použití **Studentova t-testu**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dvě hodnoty vrácené funkcí `ttest_ind` jsou:
* p-hodnota může být považována za pravděpodobnost, že dvě rozdělení mají stejný průměr. V našem případě je velmi nízká, což znamená, že existuje silný důkaz podporující, že první metaři jsou vyšší.
* t-hodnota je mezihodnota normalizovaného rozdílu průměrů, která se používá v t-testu, a porovnává se s mezní hodnotou pro danou hladinu spolehlivosti.


## Simulace normálního rozdělení s centrální limitní větou

Generátor pseudonáhodných čísel v Pythonu je navržen tak, aby nám poskytoval rovnoměrné rozdělení. Pokud chceme vytvořit generátor pro normální rozdělení, můžeme použít centrální limitní větu. Pro získání hodnoty s normálním rozdělením jednoduše spočítáme průměr vzorku vygenerovaného rovnoměrným rozdělením.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelace a Evil Baseball Corp

Korelace nám umožňuje nalézt vztahy mezi sekvencemi dat. V našem ukázkovém příkladu si představme, že existuje zlý baseballový korporát, který platí svým hráčům podle jejich výšky – čím vyšší hráč je, tím více peněz dostane. Předpokládejme základní plat 1000 dolarů a dodatečný bonus od 0 do 100 dolarů, v závislosti na výšce. Vezmeme skutečné hráče z MLB a vypočítáme jejich imaginární platy:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Nyní spočítejme kovarianci a korelaci těchto sekvencí. `np.cov` nám poskytne takzvanou **kovarianční matici**, což je rozšíření kovariance na více proměnných. Prvek $M_{ij}$ kovarianční matice $M$ je korelace mezi vstupními proměnnými $X_i$ a $X_j$, a hodnoty na diagonále $M_{ii}$ jsou rozptyly $X_{i}$. Podobně `np.corrcoef` nám poskytne **korelační matici**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelace rovná 1 znamená, že mezi dvěma proměnnými existuje silný **lineární vztah**. Lineární vztah můžeme vizuálně vidět tak, že zakreslíme jednu hodnotu proti druhé:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Podívejme se, co se stane, pokud vztah není lineární. Předpokládejme, že naše společnost se rozhodla skrýt zřejmou lineární závislost mezi výškami a platy a do vzorce zavedla nějakou nelinearitu, například `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

V tomto případě je korelace o něco menší, ale stále je docela vysoká. Nyní, aby byla vazba ještě méně zřejmá, můžeme přidat nějakou dodatečnou náhodnost přidáním náhodné proměnné k platu. Podívejme se, co se stane:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Dokážete uhodnout, proč se tečky takto srovnají do svislých linií?

Pozorovali jsme korelaci mezi uměle vytvořeným konceptem, jako je plat, a pozorovanou proměnnou *výška*. Podívejme se také, zda spolu korelují dvě pozorované proměnné, jako je výška a váha:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Bohužel jsme nedostali žádné výsledky – pouze nějaké podivné hodnoty `nan`. Je to způsobeno tím, že některé hodnoty v naší sérii nejsou definované, jsou reprezentovány jako `nan`, což způsobuje, že výsledek operace je rovněž nedefinovaný. Pohledem na matici vidíme, že problematickým sloupcem je `Weight`, protože byla spočítána samokorelace mezi hodnotami `Height`.

> Tento příklad ukazuje důležitost **přípravy dat** a **čištění**. Bez správných dat nemůžeme nic spočítat.

Použijme metodu `fillna` pro doplnění chybějících hodnot a spočítejme korelaci: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Existuje skutečně korelace, ale není tak silná jako v našem umělém příkladu. Pokud se podíváme na bodový graf jedné hodnoty vůči druhé, vztah by byl mnohem méně zřejmý:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Závěr

V tomto sešitě jsme se naučili, jak provádět základní operace s daty pro výpočet statistických funkcí. Nyní víme, jak používat pevný aparát matematiky a statistiky k ověření některých hypotéz a jak vypočítat intervaly spolehlivosti pro libovolné proměnné na základě datového vzorku.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zřeknutí se odpovědnosti**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). I když usilujeme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Originální dokument v jeho mateřském jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Nejsme odpovědní za jakákoliv nedorozumění nebo mylné výklady vyplývající z použití tohoto překladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
